# Inspect `structure.oebin` for a recording

Use this notebook to load and inspect `structure.oebin` (Open Ephys metadata).

## One file vs six probes
In Open Ephys, `structure.oebin` is typically **recording-level metadata**.
So for a recording with probes A-F, it is normal to have **one** `structure.oebin` file containing multiple continuous streams (usually one per probe stream).


In [ ]:
from pathlib import Path
import json
import pandas as pd
import sys

analysis_dir = Path.cwd().resolve()
sys.path.insert(0, str((analysis_dir / '..').resolve()))
from grant_config import load_grant_config

CONFIG_FILE = Path('../configs/grant_recording_config.json')
cfg = load_grant_config(CONFIG_FILE)
structure_oebin_path = Path(cfg['structure_oebin'])

print('Config:', cfg['config_path'])
print('Recording:', cfg['recording_name'])
print('structure.oebin:', structure_oebin_path)


In [ ]:
with structure_oebin_path.open('r', encoding='utf-8') as f:
    oebin = json.load(f)

print('Top-level keys:', list(oebin.keys()))
continuous = oebin.get('continuous', [])
print('Number of continuous streams:', len(continuous))


In [ ]:
rows = []
for i, stream in enumerate(continuous):
    rows.append({
        'stream_index': i,
        'stream_name': stream.get('folder_name'),
        'sample_rate': stream.get('sample_rate'),
        'num_channels': len(stream.get('channels', [])),
        'source_processor_name': (stream.get('source_processor') or {}).get('name'),
    })

streams_df = pd.DataFrame(rows)
display(streams_df)


In [ ]:
# Optional: verify expected probe streams from config are present
expected_streams = set(cfg['probe_stream_names'].values())
observed_streams = set(streams_df['stream_name'].dropna().astype(str))

missing = sorted(expected_streams - observed_streams)
extra = sorted(observed_streams - expected_streams)

print('Expected streams in config:', sorted(expected_streams))
print('Observed streams in structure.oebin:', sorted(observed_streams))
print('Missing expected streams:', missing if missing else 'None')
print('Extra streams not in probe_stream_names:', extra if extra else 'None')


If all expected probe streams appear here, one `structure.oebin` is correctly describing the full multi-probe recording.
